In [ ]:
# Generates Dummy json file in Files/

# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import random
from datetime import datetime, timedelta

# Initialize Spark session (if not already initialized)
spark = SparkSession.builder.appName("GenerateRandomData").getOrCreate()

# Function to generate random data
def generate_random_data(num_entries):
    data = []
    for i in range(1, num_entries + 1):
        name = f"User{i}"
        entry = {
            "id": i,
            "name": name,
            "age": random.randint(18, 65),
            "email": f"{name.lower()}@example.com",
            "created_at": (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d %H:%M:%S")
        }
        data.append(entry)
    return data

# Generate 10 random entries
random_data = generate_random_data(10)

# Define schema for the DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("created_at", StringType(), True)
])

# Create a DataFrame from the random data
df_random_data = spark.createDataFrame(random_data, schema=schema)

# Write the DataFrame to the Lakehouse in the specified path
output_path = "abfss://{WORKSPACE-NAME}@onelake.dfs.fabric.microsoft.com/raw_Bronze.Lakehouse/Files/random_data" # Replace {WORKSPACE-NAME}
df_random_data.write.format("delta").mode("overwrite").save(output_path)

print(f"Random data has been saved to the Lakehouse at '{output_path}'.")